In [1]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [2]:
loader = PDFPlumberLoader(r"D:\Petramount\Courses\AI\LLM\Basic_Home_Remedies.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[0].page_content

Number of pages in the PDF: 3


'Basic Home Remedies & Healthy Lifestyle\nGuide\n1. Introduction\nHome remedies have been used for centuries to treat common ailments using natural ingredients.\nCombined with a healthy lifestyle, these remedies can support overall well-being and disease\nprevention.\n2. Home Remedies for Common Ailments\n2.1 Cold & Flu\n\uf0b7 Ginger Tea: Boil ginger slices in water, add honey and lemon. Helps relieve congestion\nand sore throat.\n\uf0b7 Turmeric Milk: Mix 1 tsp turmeric in warm milk. Acts as an anti-inflammatory and\nboosts immunity.\n\uf0b7 Steam Inhalation: Boil water, add eucalyptus oil, and inhale steam to clear nasal\ncongestion.\n2.2 Cough\n\uf0b7 Honey & Lemon: Mix 1 tbsp honey with warm water and lemon juice. Soothes the\nthroat and reduces cough.\n\uf0b7 Tulsi (Holy Basil) Tea: Boil tulsi leaves in water and drink. Helps with respiratory\nissues.\n2.3 Headache\n\uf0b7 Peppermint Oil: Apply a few drops on the temples for relief.\n\uf0b7 Ginger Paste: Make a paste of ginger an

In [3]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

c:\Users\deepa\.conda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Number of chunks created: ", len(documents))

print(documents[1].page_content)

Number of chunks created:  7
 Baking Soda: Mix 1/2 tsp in warm water and drink for instant relief from acidity.  Apple Cider Vinegar: Take 1 tbsp in a glass of water before meals to aid digestion. 2.5 Constipation



In [5]:
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "llama-3.2-1b-instruct")



In [ ]:
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. You are an helpful assitant 
2. Use the following pieces of context to answer the question at the end.
3. Answer only by using the context and articulate it better
4. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Answer to the question:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

In [16]:
print(qa("i am not sleeping. what to drink to promote sleep?")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. Answer only by using the context and articulate it better

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content: Warm Lemon Water: Drink on an empty stomach to stimulate digestion.  Flaxseeds: Soak overnight and consume in the morning for fiber intake.  Aloe Vera Juice: Helps in smooth bowel movements. 2.6 Skin Problems (Acne, Dry Skin, Sunburn)
 Aloe Vera Gel: Apply fresh aloe vera gel to reduce acne and soothe burns.  Coconut Oil: Moisturizes dry skin and prevents infections.  Cucumber & Honey Face Pack: Hydrates skin and reduces irritation. 2.7 Joint Pain & Inflammation
 Turmeric & Black Pepper: Mix 1 tsp turmeric with a pinch of black pepper in warm
milk. Reduces inflammation.  Epsom Salt Bath: Soak in warm water with Epsom salt for muscle relaxation.  Ginger Tea: Acts as

c:\Users\deepa\.conda\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
